In [1]:
import sys
sys.path.append('../../Modules/')
sys.path.append('../../Modules/Processors from Prof')
from Packages import *
from My_CSV_processor import *

# Import processors from Prof
from ipynb.fs.full.Utilities import *
from ipynb.fs.full.Json_Processor import *
from ipynb.fs.full.CSV_Processor import *

In [2]:
boolean_df = FarelBench_CSV_Processor_my('Farel-Bench', 'farel_bench.csv').convert_df()

In [3]:
boolean_df

,topic,input,target
0,child,"Given the family relationships:\n* Ralph is Anthony's parent.\n* Albert is Ralph's parent.\nWhat is Anthony's relationship to Ralph?\nSelect the correct answer:\n1. Anthony is Ralph's child.\n2. Anthony is Ralph's parent.\nEnclose the selected answer number in the <ANSWER> tag, for example: <ANSWER>1</ANSWER>.",1
1,child,"Given the family relationships:\n* Jessica is John's parent.\n* John is Lawrence's parent.\nWhat is Lawrence's relationship to John?\nSelect the correct answer:\n1. Lawrence is John's parent.\n2. Lawrence is John's child.\nEnclose the selected answer number in the <ANSWER> tag, for example: <ANSWER>1</ANSWER>.",1
2,child,"Given the family relationships:\n* Raymond is William's parent.\n* Denise is Raymond's parent.\nWhat is William's relationship to Raymond?\nSelect the correct answer:\n1. William is Raymond's parent.\n2. William is Raymond's child.\nEnclose the selected answer number in the <ANSWER> tag, for example: <ANSWER>1</ANSWER>.",1
3,child,"Given the family relationships:\n* Samantha is Elijah's parent.\n* Elijah is Joshua's parent.\nWhat is Joshua's relationship to Elijah?\nSelect the correct answer:\n1. Joshua is Elijah's parent.\n2. Joshua is Elijah's child.\nEnclose the selected answer number in the <ANSWER> tag, for example: <ANSWER>1</ANSWER>.",1
4,child,"Given the family relationships:\n* Anna is Charlotte's parent.\n* Marie is Anna's parent.\nWhat is Charlotte's relationship to Anna?\nSelect the correct answer:\n1. Charlotte is Anna's parent.\n2. Charlotte is Anna's child.\nEnclose the selected answer number in the <ANSWER> tag, for example: <ANSWER>1</ANSWER>.",1
...,...,...,...
995,3rd great grandparent,Given the family relationships:\n* Karen is Kathryn's parent.\n* William is Jacqueline's parent.\n* Keith is Kelly's parent.\n* Linda is Andrea's parent.\n* Alexis is Walter's parent.\n* Ashley is William's parent.\n* Alexis is Kathleen's parent.\n* Randy is Victoria's parent.\n* Karen is Samantha's parent.\n* Victoria is Grace's parent.\n* Andrea is Keith's parent.\n* Walter is Roy's parent.\n* Walter is Randy's parent.\n* Grace is Shirley's parent.\n* Anthony is Linda's parent.\n* Samantha is Ashley's parent.\n* Roy is Scott's parent.\n* Kayla is Alexis' parent.\n* Kathryn is Anthony's parent.\n* Randy is Karen's parent.\nWhat is Kayla's relationship to Kathryn?\nSelect the correct answer:\n1. Kayla is Kathryn's great grand-niece or great grand-nephew.\n2. Kayla is Kathryn's 1st cousin 1x removed.\n3. Kayla is Kathryn's 3rd great grandchild.\n4. Kayla is Kathryn's 1st cousin 1x removed.\n5. Kayla is Kathryn's 3rd great grandparent.\n6. Kayla is Kathryn's great grand-aunt or great...,1
996,3rd great grandparent,Given the family relationships:\n* Ann is Juan's parent.\n* Austin is Judith's parent.\n* Steven is Cynthia's parent.\n* Amanda is Albert's parent.\n* Debra is Amanda's parent.\n* Keith is Steven's parent.\n* Raymond is Willie's parent.\n* Raymond is Terry's parent.\n* Judith is Gloria's parent.\n* Noah is Ann's parent.\n* Cynthia is Austin's parent.\n* Gabriel is Zachary's parent.\n* Terry is Diane's parent.\n* Samuel is Brandon's parent.\n* Brandon is Anthony's parent.\n* Willie is Gabriel's parent.\n* Diane is Debra's parent.\n* Ann is Samuel's parent.\n* Samuel is Raymond's parent.\n* Terry is Keith's parent.\nWhat is Noah's relationship to Keith?\nSelect the correct answer:\n1. Noah is Keith's great grand-aunt or great grand-uncle.\n2. Noah is Keith's 1st cousin 1x removed.\n3. Noah is Keith's 3rd great grandparent.\n4. Noah is Keith's 3rd great grandchild.\n5. Noah is Keith's 1st cousin 1x removed.\n6. Noah is Keith's great grand-niece or great grand-nephew.\nEnclose the sele...,1
997,3rd great grandparent,Given the family relationships:\n* Joan is Stephanie's parent.\n* Gabriel is Bobby's parent.\n* Jonathan is Patricia's parent.\n* Walter is Lori's parent.\n* Lori is Sharon's parent.\n* Lawrence is Kathryn's parent.\n* Daniel is Gabriel's parent.\

In [4]:
boolean_df['target'].iloc[3,]

'1'